In [29]:
import kfp.dsl as dsl

In [30]:
@dsl.pipeline(
    name="Volume Op DAG",
    description="The second example of the design doc."
)
def volume_op_dag():    
    vopA = dsl.VolumeOp(
        name="create_pvca",
        resource_name="my-pvca",
        size="10Gi",
        modes=dsl.VOLUME_MODE_RWO
    )

    step1 = dsl.ContainerOp(
        name="step1",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["echo 1 | tee /mnt/file_test_1"],
        pvolumes={"/mnt": vopA.volume}
    )

    step2 = dsl.ContainerOp(
        name="step2",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["echo 2 | tee /mnt2/file_test_2"],
        pvolumes={"/mnt2": vopA.volume}
    )

    step3 = dsl.ContainerOp(
        name="step3",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["cat /mnt/file1 /mnt/file2 /mnt/file_test_1 /mnt/file_test_2"],
        pvolumes={"/mnt": vopA.volume.after(step1, step2)}
    )

In [31]:
import kfp
import kfp.compiler as compiler
compiler.Compiler().compile(volume_op_dag, "exemplo_volume" + ".tar.gz")

In [32]:
run_result = kfp.Client(host='http://localhost:8080/pipeline').create_run_from_pipeline_package('exemplo_volume.tar.gz', arguments={},)